Prueba 1: Check non numerica elements 

In [4]:
import requests
import time
import json
from pymongo import MongoClient
import pandas as pd
import os
from bson.regex import Regex

In [12]:
client = MongoClient("localhost:27017")
db = client["hsz"]
c = db.get_collection("hardiness_zones_coordinates")
pipeline = [
    {
        "$match": {
            "fieldToCheck": {"$type": "string"}  # Assuming the fieldToCheck contains strings
        }
    },
    {
        "$match": {
            "fieldToCheck": {"$not": Regex(r'^[-+]?[0-9]*\.?[0-9]+$')}  # Regular expression to match non-numeric values
        }
    }
]
documents = c.aggregate(pipeline)

document = list(documents)
document_count = len(document)

if document_count == 0:
    print("No non-numeric values found.")
else:
    print(f"Found {document_count} non-numeric value(s).")


No non-numeric values found.


In [10]:
print(documents)


In [65]:
client = MongoClient("localhost:27017")
db = client["hsz"]
c = db.get_collection("hardiness_zones_coordinates")


In [66]:
long = 0.666
lat = 42.633

In [67]:
my_position = {"type": "Point", "coordinates": [long, lat]}

In [69]:
c.create_index([('geometry', '2dsphere')])

print('2D sphere index created successfully.')


2D sphere index created successfully.


In [78]:
result = c.find(
            {"geometry": 
                     {"$geoIntersects": 
                         {"$geometry": my_position}}
            })
print(result)

TypeError: index 'MultyPolygon' cannot be applied to Cursor instances

PROVA 2: 5A

In [64]:
import geojson
def fix_polygon_winding_order(geometry):
    if isinstance(geometry, geojson.Polygon):
        # Fix winding order for a single Polygon
        geometry.coordinates = fix_ring_winding_order(geometry.coordinates)
    elif isinstance(geometry, geojson.MultiPolygon):
        # Fix winding order for each Polygon within the MultiPolygon
        for polygon in geometry.coordinates:
            polygon[0] = fix_ring_winding_order(polygon[0])
            for interior_ring in polygon[1:]:
                interior_ring = fix_ring_winding_order(interior_ring)
    else:
        raise ValueError("Invalid geometry type. Only Polygon and MultiPolygon are supported.")

def fix_ring_winding_order(coordinates):
    ring_area = calculate_ring_area(coordinates)
    if ring_area < 0:
        # Reverse the coordinates to fix the winding order
        coordinates = coordinates[::-1]
    return coordinates

def calculate_ring_area(coordinates):
    area = 0.0
    for i in range(len(coordinates) - 1):
        lon1, lat1 = coordinates[i]
        lon2, lat2 = coordinates[i + 1]
        area += (lon2 - lon1) * (lat2 + lat1)
    return area

# Load the GeoJSON data
with open('C:/Users/manya/Documents/Ironhack/Course/Final-project/geojson/old_geojson/ES_HZ_5a.geojson') as f:
    data = geojson.load(f)

# Fix the winding order for each geometry in the GeoJSON
for feature in data.features:
    fix_polygon_winding_order(feature.geometry)

# Save the modified GeoJSON to a new file
with open('output.geojson', 'w') as f:
    geojson.dump(data, f)

Prova sense mongo

In [100]:
from shapely.geometry import Point,MultiPolygon, shape

# Coordenadas de la ubicación que deseas verificar
coordenadas_punto = (-3.399,37.033)

# GeoJSON del multipolígono


file_path = os.path.join('C:/Users/manya/Documents/Ironhack/Course/Final-project/geojson/old_geojson/ES_HZ_5a_Copy.geojson')
            # Load the GeoJSON file
with open(file_path) as file:
    geojson_multipolygon = json.load(file)
print(geojson_multipolygon)

# Crear un objeto de punto a partir de las coordenadas
punto = Point(coordenadas_punto)
print(punto)
# Crear una geometría a partir del GeoJSON del multipolígono
multipoligono = MultiPolygon([shape(polygon) for polygon in geojson_multipolygon['coordinates']])
print(multipoligono)

# Verificar si el punto se encuentra dentro del multipolígono
if multipoligono.contains(punto):
    print("El punto está dentro del multipolígono.")
else:
    print("El punto NO está dentro del multipolígono.")

{'type': 'MultiPolygon', 'coordinates': [[[[-3.399, 37.033], [-3.399, 37.0], [-3.433, 37.0], [-3.433, 37.033], [-3.399, 37.033]]], [[[-3.399, 37.033], [-3.399, 37.066], [-3.366, 37.066], [-3.333, 37.066], [-3.299, 37.066], [-3.299, 37.1], [-3.266, 37.1], [-3.266, 37.066], [-3.266, 37.033], [-3.299, 37.033], [-3.333, 37.033], [-3.366, 37.033], [-3.399, 37.033]]], [[[-3.266, 37.1], [-3.266, 37.133], [-3.233, 37.133], [-3.233, 37.1], [-3.266, 37.1]]], [[[2.233, 42.426], [2.233, 42.4], [2.2, 42.4], [2.166, 42.4], [2.133, 42.4], [2.133, 42.366], [2.1, 42.366], [2.1, 42.378], [2.1, 42.378], [2.102, 42.379], [2.104, 42.38], [2.104, 42.38], [2.105, 42.38], [2.106, 42.38], [2.106, 42.38], [2.108, 42.38], [2.109, 42.38], [2.112, 42.381], [2.114, 42.381], [2.115, 42.382], [2.116, 42.383], [2.116, 42.383], [2.116, 42.383], [2.115, 42.384], [2.115, 42.384], [2.115, 42.385], [2.115, 42.386], [2.115, 42.387], [2.114, 42.389], [2.114, 42.389], [2.114, 42.391], [2.114, 42.391], [2.114, 42.392], [2.114,

AttributeError: 'list' object has no attribute 'get'